In [45]:
import torch
from PIL import Image
import requests
import torchvision.transforms as T
import matplotlib.pyplot as plt
from collections import defaultdict
import torch.nn.functional as F
import numpy as np
from skimage.measure import find_contours

from matplotlib import patches,  lines
from matplotlib.patches import Polygon

torch.set_grad_enabled(False);

In [46]:
%matplotlib notebook
%matplotlib inline

VQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/VQA'
GQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/GQA'

import pandas as pd
import json
import pickle as pkl
import numpy as np
import h5py
import random
from PIL import Image
import IPython.display as display
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import torch

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [47]:
qid_to_prompts={}

with open('../clip_vqa/yesno_prompt.txt') as number:
    for line in number:
        line = line.strip().split('\t')
        qid, quest, prompt = line[0], line[1].lower(), json.loads(line[2])
        qid_to_prompts[qid] = prompt
        
print (len(qid_to_prompts))
names=list(qid_to_prompts.keys())

80541


In [48]:
print(qid_to_prompts[names[0]][0])

this is a creamy soup


In [49]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [50]:
import inflect
stemmer = inflect.engine()

In [51]:
val_all=load_json(os.path.join(VQA_PATH,'scenes','val_all.json'))
print ('Length of testing data:',len(val_all))

Length of testing data: 214354


In [8]:
import skimage
from skimage import io as skimage_io
from skimage import transform as skimage_transform

def get_vqa_feat(img_id):
    # Load example image:
    filename = os.path.join(VQA_PATH,'images/val2014',img_id)
    image_uint8 = skimage_io.imread(filename)
    image = image_uint8.astype(np.float32) / 255.0

    # Pad to square with gray pixels on bottom and right:
    h, w, _ = image.shape
    size = max(h, w)
    image_padded = np.pad(
        image, ((0, size - h), (0, size - w), (0, 0)), constant_values=0.5)

    # Resize to model input size:
    input_image = skimage.transform.resize(
        image_padded,
        (840, 840),
        anti_aliasing=True)
    return input_image

In [9]:
import clip

CUDA_DEVICE=1
torch.cuda.set_device(CUDA_DEVICE)
device = torch.device("cuda:"+str(CUDA_DEVICE))
#the default ipykernel links to the first conda environment

model, postprocessor = torch.hub.load('ashkamath/mdetr:main', 'mdetr_efficientnetB5', pretrained=True, return_postprocessor=True)
model = model.to(device)
model.eval();

clip_model, preprocess = clip.load("ViT-B/32",device=device)

Using cache found in /root/.cache/torch/hub/ashkamath_mdetr_main
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def plot_inference(im, caption, plot=True):
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0).cuda()

    # propagate through the model
    memory_cache = model(img, [caption], encode_and_save=True)
    outputs = model(img, [caption], encode_and_save=False, memory_cache=memory_cache)
    #print (outputs['pred_logits'])

    # keep only predictions with 0.7+ confidence
    probas = 1 - outputs['pred_logits'].softmax(-1)[0, :, -1].cpu()
    keep = (probas > 0.7).cpu()

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'].cpu()[0, keep], im.size)

    # Extract the text spans predicted by each box
    positive_tokens = (outputs["pred_logits"].cpu()[0, keep].softmax(-1) > 0.1).nonzero().tolist()
    predicted_spans = defaultdict(str)
    for tok in positive_tokens:
        item, pos = tok
        if pos < 255:
            span = memory_cache["tokenized"].token_to_chars(0, pos)
            predicted_spans [item] += " " + caption[span.start:span.end]

    labels = [predicted_spans [k] for k in sorted(list(predicted_spans .keys()))]
    if plot:
        plot_results(im, probas[keep], bboxes_scaled, labels)
    return probas[keep], bboxes_scaled.tolist(), labels

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

def plot_results(pil_img, scores, boxes, labels, masks=None):
    plt.figure(figsize=(16,10))
    np_image = np.array(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if masks is None:
        masks = [None for _ in range(len(scores))]
    assert len(scores) == len(boxes) == len(labels) == len(masks)
    for s, (xmin, ymin, xmax, ymax), l, mask, c in zip(scores, boxes.tolist(), labels, masks, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{l}: {s:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

        if mask is None:
            continue
        np_image = apply_mask(np_image, mask, c)

        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=c)
            ax.add_patch(p)


    plt.imshow(np_image)
    plt.axis('off')
    plt.show()


def add_res(results, ax, color='green'):
    #for tt in results.values():
    if True:
        bboxes = results['boxes']
        labels = results['labels']
        scores = results['scores']
        #keep = scores >= 0.0
        #bboxes = bboxes[keep].tolist()
        #labels = labels[keep].tolist()
        #scores = scores[keep].tolist()
    #print(torchvision.ops.box_iou(tt['boxes'].cpu().detach(), torch.as_tensor([[xmin, ymin, xmax, ymax]])))
    
    colors = ['purple', 'yellow', 'red', 'green', 'orange', 'pink']
    
    for i, (b, ll, ss) in enumerate(zip(bboxes, labels, scores)):
        ax.add_patch(plt.Rectangle((b[0], b[1]), b[2] - b[0], b[3] - b[1], fill=False, color=colors[i], linewidth=3))
        cls_name = ll if isinstance(ll,str) else CLASSES[ll]
        text = f'{cls_name}: {ss:.2f}'
        print(text)
        ax.text(b[0], b[1], text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

In [11]:
def vis_bbox_only(bbox,scores,input_image,gt_bbox=None,text=None,threshold=0.2,vis_pred=True):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(input_image, extent=(0, 1, 1, 0))
    ax.set_axis_off()
    if vis_pred:
        for i,box in enumerate(bbox):
            score=scores[i]
            if score<threshold:
                continue
            cx, cy, w, h = box
            ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                    [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'r')
            ax.text(
                cx - w / 2,
                cy + h / 2 + 0.015,
                f'{score:1.2f}',
                ha='left',
                va='top',
                color='red',
                bbox={
                    'facecolor': 'white',
                    'edgecolor': 'red',
                    'boxstyle': 'square,pad=.3'
                })
    if text is not None:
        ax.set_title(text,  fontsize=12)
    if gt_bbox is not None:
        for i,box in enumerate(gt_bbox):
            cx, cy, w, h = box
            ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                    [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'b')

In [12]:
import IPython.display as display

def size_aware_pred(bb):
    if bb[2]>0.3 or bb[3]>0.3:
        return 'large'
    else:
        return 'small'
def pos_aware_pred(valid_bbox,candidates):
    if 'bottom' in candidates:
        if valid_bbox[1]>0.5:
            return 'bottom'
        else:
            return 'top'
    elif 'right' in candidates:
        if valid_bbox[0]<0.5:
            return 'left'
        else:
            return 'right'

In [13]:
def transform_coord(bbox,img_id):
    im=Image.open(os.path.join(VQA_PATH,'images/val2014',img_id))
    width, height=im.size
    #width=val_graphs[img_id]['width']
    #height=val_graphs[img_id]['height']
    size = max(height, width)
    x=bbox[0]
    y=bbox[1]
    w=bbox[2]-bbox[0]
    h=bbox[3]-bbox[1]
    #print (bbox)
    return [(x+w/2)/size,(y+h/2)/size,w/size,h/size]

In [14]:
def preprocess_mdert_result(scores,bboxs,labels,obj,img_id):
    sf_s=[]
    sf_bbox=[]
    labels=[''.join(label.split(' ')) for label in labels]
    flag=False
    if obj in labels:
        flag=True
    #print (obj,labels)
    for i,label in enumerate(labels):
        if flag:
            if obj==label:
                sf_s.append(scores[i])
                sf_bbox.append(bboxs[i])
        else:
            if obj in label:
                sf_s.append(scores[i])
                sf_bbox.append(bboxs[i])
    #print (len(sf_s))
    if len(sf_bbox)==0:
        for i,label in enumerate(labels):
            sf_s.append(scores[i])
            sf_bbox.append(bboxs[i])
    max_s=max(sf_s)
    max_id=sf_s.index(max_s)
    return_bbox=sf_bbox[max_id]
    coord=transform_coord(return_bbox,img_id)
    return max_s.item(),coord
#only consider one bbox now

In [15]:
def generate_phrase_ground(img_id,cap,obj):
    cap=cap+' '
    im=Image.open(os.path.join(VQA_PATH,'images/val2014',img_id))
    try:
        scores,bboxs,labels=plot_inference(im, cap, plot=False)
    except:
        print ('Invalid caption generation for',img_id,obj,cap)
        scores=[torch.Tensor([0.99])]
        bboxs=[[0.5,0.5,0.99,0.99]]
        labels=[obj]
    if len(bboxs)==0:
        max_s=0.99
        valid_bbox=[0.5,0.5,0.99,0.99]
    else:
        # print (scores,bboxs)
        max_s,valid_bbox=preprocess_mdert_result(scores,bboxs,labels,obj,img_id)
    return valid_bbox

In [16]:
def get_cropped(bb,input_image,showing=False):
    width=bb[2]*840
    height=bb[3]*840
    xy=(bb[0]*840-width/2,bb[1]*840-height/2)
    h0=int(max(0,xy[1]))
    w0=int(max(0,xy[0]))
    h1=int(max(0,840-(height+xy[1])))
    w1=int(max(0,840-(width+xy[0])))
    cropped=skimage.util.crop(input_image,((h0,h1),(w0,w1),(0,0)), copy=False)
    trans_crop=Image.fromarray(np.uint8(cropped*255.0))
    if showing:
        display.display(trans_crop)
    return trans_crop

def clip_aware_pred(img_feat,valid_bbox,candidates,showing=False):
    #tokens=clip.tokenize(candidates)
    trans_crop=get_cropped(valid_bbox,img_feat)
    if showing:
        display.display(trans_crop)
    with torch.no_grad():
        image_features = clip_model.encode_image(preprocess(trans_crop).unsqueeze(0).to(device))
        text_features = clip_model.encode_text(clip.tokenize(candidates).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(2)
    #print (indices[0][0].item())
    ans=candidates[indices[0][0].item()]
    return ans

In [17]:
def show_img_with_text(img_path,texts):
    f = Image.open(img_path)
    img = f.convert('RGB')
    img = np.asarray(img, dtype=np.float32)
    
    fig = plt.figure()
    ax=fig.add_subplot(1,1,1)
    ax.set_title(texts,  fontsize=12)
    ax.imshow(img.astype(np.uint8))  

In [18]:
position_holder=['left of','right of','top of','bottom of',
                 'under ','below ','beneath ','underneath ']

In [19]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [20]:
def binary_matching(img_id,sents):
    tokens=clip.tokenize(sents)
    im=Image.open(os.path.join(VQA_PATH,'images/val2014',img_id))
    with torch.no_grad():
        text_features=clip_model.encode_text(tokens.to(device))
        image_features = clip_model.encode_image(preprocess(im).unsqueeze(0).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    indices=torch.max(similarity,1)[1]
    #print (similarity,indices)
    similarity=similarity.squeeze()
    scores=[str(similarity[0].item()),str(similarity[1].item())]
    if indices.squeeze().item()==0:
        pred='yes'
    else:
        pred='no'
    return pred

In [21]:
q_type_counter=defaultdict(int)

binary_q={}
for i,row in enumerate(val_all):
    if row['ans_type']!='yes/no':
        continue
    binary_q[row['idx']]={
        'question':row['question'],
        'img_id':row['img_id'],
        'answer':row['answer'],
        'scores':row['scores'],
        'q_type':row['q_type'],
        'ans_type':row['ans_type']
    }    
    q_type_counter[row['q_type']]+=1
print (q_type_counter)
print (len(binary_q))

defaultdict(<class 'int'>, {'is this a': 6921, 'is this': 7147, 'is it': 3241, 'none of the above': 4193, 'is there a': 4663, 'is the': 15510, 'is that a': 631, 'are these': 2620, 'are the': 4827, 'could': 618, 'are there': 2718, 'has': 941, 'are there any': 1330, 'is this person': 681, 'can you': 844, 'does this': 2151, 'is this an': 835, 'is there': 3097, 'are': 2313, 'do you': 713, 'is he': 1041, 'does the': 3045, 'is': 3009, 'is the woman': 937, 'are they': 1269, 'is the person': 651, 'was': 739, 'is the man': 2372, 'do': 1455, 'what': 15, 'what is the': 4, 'what is': 2, 'why': 3, 'what are': 1, 'what are the': 2, 'why is the': 1, 'how': 1})
80541


In [22]:
names=list(binary_q.keys())
print (len(names))

80541


In [23]:
parsing_file=load_pkl('../VQA_Zero-shot_decomposable/binary_parsing.pkl')
print(len(parsing_file))

80541


In [24]:
position_counter={p:0 for p in position_holder}

for name in names:
    info=binary_q[name]
    ques=info['question'].lower()[:-1]
    for p in position_holder:
        if p in ques:
            position_counter[p]+=1
            print (p,'\n\t',info['question'])
            parser=parsing_file[name]
            if parser['invalid']==False:
                layout=parser['layout']
                for step in layout:
                    print ('\t',step['operation'],' '.join(step['argument']))
                print ('\n')
print (position_counter)

right of 
	 Does the metalwork to the right of the clock resemble a birdcage?
	 select scene
	 relocate to the right of the clock
	 select metalwork
	 and 


top of 
	 Is there a mirror on top of the fireplace?
	 select scene
	 relocate on top of the fireplace
	 select mirror
	 and 


under  
	 Is there a balcony under the clock?
	 select clock
	 relocate under the clock
	 select balcony
	 and 


under  
	 Do the kites make the sky look like it's under the sea?
	 select sea
	 relocate under the sea
	 select kites
	 and 


top of 
	 Is there a man on top of the horse?
	 select scene
	 relocate on top of the horse
	 select man
	 and 


under  
	 Is the thing under the blanket alive?
	 select blanket
	 filter alive
	 relocate under the blanket alive
	 select thing
	 and 


under  
	 Is there chocolate sauce under the cake?
	 select cake
	 relocate under the cake
	 select chocolate sauce
	 and 


top of 
	 Is that marshmallows on top of the pizza?
	 select scene
	 relocate on top of the pi

	 filter many
	 filter too
	 select scene


under  
	 Is the baby under the adult?
	 select adult
	 relocate under the adult
	 select baby
	 and 


under  
	 Are the people standing under a dock?
	 select dock
	 relocate under a dock
	 filter standing
	 select people
	 and 


left of 
	 Is the sink to the left of the toilet in the photo?
	 select scene
	 relocate to the left of the toilet
	 select sink
	 and 


under  
	 Are the girls under the umbrella wearing the same dresses?
	 select scene
	 relocate under the umbrella
	 select girls
	 and 


below  
	 Are there flames shooting out from the 3 spear looking things below the clock?
	 select flames


bottom of 
	 Are there at least two other ways someone could reach the bottom of this hill?
	 select hill
	 relocate of this hill
	 select ways
	 and 
	 filter least


under  
	 Can you walk under the clock?
	 select clock
	 relocate under the clock
	 select can
	 and 


top of 
	 Are the portals at the top of the tower arched?
	 select s

In [25]:
qid_to_prompt={}
with open('../clip_vqa/yesno_prompt.txt') as number:
    for line in number:
        line = line.strip().split('\t')
        qid, quest, prompt = line[0], line[1].lower(), json.loads(line[2])
        qid_to_prompt[qid]=prompt
print (len(qid_to_prompt))

80541


In [26]:
random.shuffle(names)
vis=0
op_seq_dict=defaultdict(int)
for name in names:
    info=binary_q[name]
    ques=info['question'].lower()[:-1]
    vis+=1
    parser=parsing_file[name]
    if parser['invalid']:
        continue
    else:
        layout=parser['layout']
        op_seq_dict['-'.join(step['operation'] for step in layout)]+=1

In [27]:
def binary_matching(img_id,sents):
    tokens=clip.tokenize(sents)
    im=Image.open(os.path.join(VQA_PATH,'images/val2014',img_id))
    with torch.no_grad():
        text_features=clip_model.encode_text(tokens.to(device))
        image_features = clip_model.encode_image(preprocess(im).unsqueeze(0).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    indices=torch.max(similarity,1)[1]
    similarity=similarity.squeeze()
    scores=[str(similarity[0].item()),str(similarity[1].item())]
    if indices.squeeze().item()==0:
        pred='yes'
    else:
        pred='no'
    return pred

In [28]:
def cropped_pred(img_id,valid_bbox,candidates,showing=False):
    img_feat=get_vqa_feat(img_id)
    trans_crop=get_cropped(valid_bbox,img_feat)
    if showing:
        display.display(trans_crop)
    with torch.no_grad():
        image_features = clip_model.encode_image(preprocess(trans_crop).unsqueeze(0).to(device))
        text_features = clip_model.encode_text(clip.tokenize(candidates).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(2)
    if indices[0][0].item()==0:
        pred='yes'
    else:
        pred='no'
    return pred

In [29]:
def size_aware_pred(bb):
    if bb[2]>0.3 or bb[3]>0.3:
        return 'big'#long
    else:
        return 'small'#short

In [39]:
def bbox_generator(img_id, idx, arg):
    if arg in hold_list:
        return [0.5,0.5,0.99,0.99]
    obj=stemmer.singular_noun(arg)
    if obj==False:
        obj=arg
    try:
        scenic_result=load_pkl(os.path.join(
            '../Learning-Asking/prompt-clip/scenic/all_binary',
            idx+'.pkl'))
        scores=[s['score'] for s in scenic_result[obj]]
        bboxs=[s['bbox'] for s in scenic_result[obj]]
    except:
        #print ('No exisiting scenic for %s % s' % 
        #       (img_id,obj))
        bboxs=[]
        
    if len(bboxs)>0:
        max_s=max(scores)
        max_idx=scores.index(max_s)
        bbox=bboxs[max_idx]
        cap=obj
    else:
        cap='the '+obj
        split_obj=''.join(cap.split(' '))
        bbox=generate_phrase_ground(img_id,cap,split_obj)
        
    return bbox

In [31]:
hold_list=['back','front','left','right','hot','position','plenty',
            'behind','above','below','top','bottom','over','years',
            'moving','part','weather',
            'beneath','underneath','lot','way','small','large','side',
            'other','looking','most','something','anything','definition',
            'is','scene','does','are']

In [32]:
def direct_check(idx,arg,threshold=0.2):    
    if arg in hold_list:
        return 'yes'
    obj=stemmer.singular_noun(arg)
    if obj==False:
        obj=arg
    if obj in ['person','animal','item','someone','anyone']:
        threshold=0.13
    scenic_file=load_pkl(
        os.path.join(
            '../Learning-Asking/prompt-clip/scenic/all_binary',
            idx+'.pkl')
    )
    info=scenic_file[obj]
    for row in info:
        if row['score']>=threshold:
            return 'yes'
    return 'no'

In [33]:
def pred_decomp_binary(idx,ques,img_id,layout):
    ops='-'.join(step['operation'] for step in layout)
    if ops not in valid_ops:
        #print ('Prompt matching')
        prompts=qid_to_prompt[idx]
        pred=binary_matching(img_id,prompts)
    elif ops in ['select','select-select-and']:
        #print ('Direct checking',idx)
        obj=layout[0]['argument'][0]
        pred=direct_check(idx,obj)
        if ops=='select':
            return pred
        else:
            if pred=='no':
                return pred
            else:
                pred_sec=direct_check(idx,layout[1]['argument'][0])
                if pred_sec=='yes':
                    return 'yes'
                else:
                    return 'no'
    elif ops.endswith('-filter'):
        #print ('Filter checking')
        pred=filter_check(idx,img_id,layout)
    elif ops.endswith('-and'):
        #print ('And checking')
        pred=and_check(idx,img_id,layout)
    return pred

In [34]:
valid_ops=[
    'select',
    'select-select-and',
    
    'select-relocate-select-and',
    'select-filter-select-and',
    'select-relocate-filter-select-and',
    'select-filter-relocate-select-and',
    'select-filter-filter-select-and',
    'select-relocate-relocate-select-and',
    
    'select-filter',
    'select-relocate-select-and-filter',
    'select-select-and-filter',
]

In [86]:
def pred_all_binary(parser,idx,ques,img_id,q_t):
    invalid=parser['invalid']
    if q_t in current_focus:
        pred=there_related_pred(idx)
    elif invalid:
        prompts=qid_to_prompt[idx]
        pred=binary_matching(img_id,prompts)
    else:
        layout=parser['layout']
        #print ('Layout')
        #for step in layout:
        #    print ('\t',step['operation'],' '.join(step['argument']))
        pred=pred_decomp_binary(idx,ques,img_id,layout)
    return pred

In [98]:
def there_related_pred(name):
    arg=obj_needed_there[name]
    obj=stemmer.singular_noun(arg)
    if obj==False:
        obj=arg
    try:
        scenic_info=load_pkl(
            os.path.join(
                '../Learning-Asking/prompt-clip/scenic/there_related',
                name+'.pkl'))
        #print(scenic_info,obj)
        bbox=[info['score'] for info in scenic_info[obj] if info['score']>=0.2]
    except:
        bbox=[]
    #print (bbox)
    if len(bbox)>0:
        pred='yes'
    else:
        pred='no'
    return pred

In [99]:
def filter_check(idx,img_id,layout):
    obj_list=[]
    length=len(layout)
    #print (length)
    adj=layout[-1]['argument'][0]
    for i in range(length):
        row=layout[length-i-1]
        if i==0:
            continue
        #print(row,length-i)
        
        op=row['operation']
        arg=row['argument']
        if op=='relocate':
            obj_list.append(arg[0])
            break
        elif op!='and':
            obj_list.append(arg[0])
    
    if len(obj_list)==1:
        valid_bbox=bbox_generator(img_id,idx,layout[0]['argument'][0])
    else:
        cap=' '.join(obj_list)
        valid_bbox=generate_phrase_ground(img_id,cap,obj_list[-1])
    #print (obj_list,adj)
    if adj in ['big','small']:
        ans=size_aware_pred(valid_bbox)
        if ans==adj:
            pred='yes'
        else:
            pred='no'
    else:
        pred=cropped_pred(img_id,valid_bbox,[adj,'Not '+adj])
        
    return pred

In [100]:
def and_check(idx,img_id,layout):
    obj_verify=direct_check(idx,layout[-2]['argument'][0])
    if obj_verify=='no':
        return obj_verify
    valid_bbox=bbox_generator(img_id,idx,layout[-2]['argument'][0])
    length=len(layout)
    adj_list=[]
    filter_arg=[]
    #print ('Length',length)
    for i in range(length):
        row=layout[length-i-1]
        #print (i,row)
        if i<=1:
            continue
        #print (row)
        op=row['operation']
        arg=row['argument']
        if op=='relocate':
            adj_list.append(arg[0])
            break
        else:
            adj_list.append(arg[0])
            if op=='filter':
                filter_arg.append(arg[0])
    adj=' '.join(adj_list)
    #print (layout[-2]['argument'],adj)
    if 'wearing' in filter_arg:
        pred=direct_check(idx,layout[0]['argument'][0])
    else:
        pred=cropped_pred(img_id,valid_bbox,[adj,'Not '+adj])
    return pred

In [52]:
current_focus=['is there a', 'is there', 'are there any', 'are there']

In [103]:
random.shuffle(names)
vis=0
acc={q:0.0 for q in q_type_counter}
counter={q:0.0 for q in q_type_counter}
invalid=[]
for name in names:
    #if vis>10:
    #    break
    info=binary_q[name]
    ques=info['question'].lower()[:-1]
    
    parser=parsing_file[name]
    """if parser['invalid']:
        continue
    else:
        layout=parser['layout']
        ops='-'.join(step['operation'] for step in layout)
        if ops not in valid_op_seq:
            continue
        if ops not in ['select-relocate-select-and-filter']:
            continue
        print ('\n',ques,'\n',info['q_type'])
        print (qid_to_prompt[name])
        for step in layout:
            print ('\t',step['operation'],' '.join(step['argument']))"""
    img_id='COCO_val2014_'+str(info['img_id']).zfill(12)+'.jpg'
    #print (ques,'\t',name,'\n',info['q_type'],info['answer'])
    #print (qid_to_prompt[name])
    #pred=pred_all_binary(parser,name, ques,img_id,info['q_type'])
    try:
        pred=pred_all_binary(parser,name, ques,img_id,info['q_type'])
    except:
        invalid.append(name)
        continue
    #print ('\tPrediction:',pred,'\n')
    vis+=1
    
    counter[info['q_type']]+=1
    if pred in info['answer']:
        score=info['scores'][info['answer'].index(pred)]
        acc[info['q_type']]+=score
        
    if vis%1000==0:
        print('Already finish:',vis,len(invalid))
        for q_t in counter:
            if counter[q_t]==0:
                continue
            print ('\tQuestion type:%s, Accuracy %.2f' % 
                   (q_t,acc[q_t]*100.0/counter[q_t]))
        print('\tAccuracy %.2f\n' % 
              (sum(acc.values())*100.0/sum(counter.values())))  

print ('Overall')
for q_t in counter:
    if counter[q_t]==0:
        continue
    print ('\tQuestion type:%s, Accuracy %.2f, Number of instance %d' % 
           (q_t,acc[q_t]*100.0/counter[q_t],counter[q_t]))
print('\tAccuracy %.2f' % 
      (sum(acc.values())*100.0/sum(counter.values())))  
print ('Invalid',len(invalid))

Invalid caption generation for COCO_val2014_000000024021.jpg theschool the school 
Already finish: 1000 5
	Question type:is this a, Accuracy 73.48
	Question type:is this, Accuracy 68.99
	Question type:is it, Accuracy 82.22
	Question type:none of the above, Accuracy 67.45
	Question type:is there a, Accuracy 84.51
	Question type:is the, Accuracy 63.44
	Question type:is that a, Accuracy 100.00
	Question type:are these, Accuracy 78.52
	Question type:are the, Accuracy 67.76
	Question type:could, Accuracy 98.00
	Question type:are there, Accuracy 72.43
	Question type:has, Accuracy 76.67
	Question type:are there any, Accuracy 89.29
	Question type:is this person, Accuracy 50.00
	Question type:can you, Accuracy 82.00
	Question type:does this, Accuracy 67.14
	Question type:is this an, Accuracy 68.00
	Question type:is there, Accuracy 73.33
	Question type:are, Accuracy 75.81
	Question type:do you, Accuracy 80.00
	Question type:is he, Accuracy 75.71
	Question type:does the, Accuracy 55.38
	Question 

Already finish: 8000 21
	Question type:is this a, Accuracy 74.60
	Question type:is this, Accuracy 73.65
	Question type:is it, Accuracy 74.40
	Question type:none of the above, Accuracy 68.16
	Question type:is there a, Accuracy 78.34
	Question type:is the, Accuracy 67.44
	Question type:is that a, Accuracy 76.82
	Question type:are these, Accuracy 72.53
	Question type:are the, Accuracy 70.37
	Question type:could, Accuracy 79.25
	Question type:are there, Accuracy 72.01
	Question type:has, Accuracy 72.47
	Question type:are there any, Accuracy 77.01
	Question type:is this person, Accuracy 67.50
	Question type:can you, Accuracy 66.59
	Question type:does this, Accuracy 67.59
	Question type:is this an, Accuracy 68.12
	Question type:is there, Accuracy 68.58
	Question type:are, Accuracy 69.91
	Question type:do you, Accuracy 81.76
	Question type:is he, Accuracy 61.94
	Question type:does the, Accuracy 68.94
	Question type:is, Accuracy 69.68
	Question type:is the woman, Accuracy 68.91
	Question type:

Invalid caption generation for COCO_val2014_000000386204.jpg thewomanfingernail the woman fingernail 
Already finish: 14000 45
	Question type:is this a, Accuracy 75.29
	Question type:is this, Accuracy 72.12
	Question type:is it, Accuracy 75.11
	Question type:none of the above, Accuracy 69.36
	Question type:is there a, Accuracy 78.48
	Question type:is the, Accuracy 67.45
	Question type:is that a, Accuracy 78.60
	Question type:are these, Accuracy 70.43
	Question type:are the, Accuracy 69.93
	Question type:could, Accuracy 72.50
	Question type:are there, Accuracy 72.90
	Question type:has, Accuracy 67.82
	Question type:are there any, Accuracy 78.05
	Question type:is this person, Accuracy 70.79
	Question type:can you, Accuracy 66.41
	Question type:does this, Accuracy 69.73
	Question type:is this an, Accuracy 66.65
	Question type:is there, Accuracy 67.86
	Question type:are, Accuracy 66.56
	Question type:do you, Accuracy 79.36
	Question type:is he, Accuracy 66.61
	Question type:does the, Accur

Already finish: 20000 55
	Question type:is this a, Accuracy 73.85
	Question type:is this, Accuracy 72.81
	Question type:is it, Accuracy 74.88
	Question type:none of the above, Accuracy 69.32
	Question type:is there a, Accuracy 78.42
	Question type:is the, Accuracy 67.73
	Question type:is that a, Accuracy 76.92
	Question type:are these, Accuracy 70.14
	Question type:are the, Accuracy 69.43
	Question type:could, Accuracy 68.84
	Question type:are there, Accuracy 72.50
	Question type:has, Accuracy 66.83
	Question type:are there any, Accuracy 78.35
	Question type:is this person, Accuracy 68.49
	Question type:can you, Accuracy 68.37
	Question type:does this, Accuracy 69.60
	Question type:is this an, Accuracy 69.51
	Question type:is there, Accuracy 68.61
	Question type:are, Accuracy 66.71
	Question type:do you, Accuracy 73.97
	Question type:is he, Accuracy 65.46
	Question type:does the, Accuracy 66.35
	Question type:is, Accuracy 69.92
	Question type:is the woman, Accuracy 67.02
	Question type

Invalid caption generation for COCO_val2014_000000443689.jpg themansarm the mans arm 
Already finish: 26000 70
	Question type:is this a, Accuracy 74.43
	Question type:is this, Accuracy 72.31
	Question type:is it, Accuracy 75.57
	Question type:none of the above, Accuracy 69.14
	Question type:is there a, Accuracy 77.61
	Question type:is the, Accuracy 67.35
	Question type:is that a, Accuracy 77.16
	Question type:are these, Accuracy 70.12
	Question type:are the, Accuracy 68.64
	Question type:could, Accuracy 69.11
	Question type:are there, Accuracy 72.13
	Question type:has, Accuracy 65.93
	Question type:are there any, Accuracy 78.17
	Question type:is this person, Accuracy 68.82
	Question type:can you, Accuracy 68.43
	Question type:does this, Accuracy 68.60
	Question type:is this an, Accuracy 70.93
	Question type:is there, Accuracy 69.43
	Question type:are, Accuracy 65.91
	Question type:do you, Accuracy 76.25
	Question type:is he, Accuracy 64.31
	Question type:does the, Accuracy 66.79
	Quest

...

Invalid caption generation for COCO_val2014_000000421195.jpg thefarm the farm 
Already finish: 56000 129
	Question type:is this a, Accuracy 73.63
	Question type:is this, Accuracy 71.81
	Question type:is it, Accuracy 75.15
	Question type:none of the above, Accuracy 68.10
	Question type:is there a, Accuracy 77.42
	Question type:is the, Accuracy 67.28
	Question type:is that a, Accuracy 74.96
	Question type:are these, Accuracy 69.89
	Question type:are the, Accuracy 67.42
	Question type:could, Accuracy 68.73
	Question type:are there, Accuracy 70.99
	Question type:has, Accuracy 65.27
	Question type:are there any, Accuracy 76.43
	Question type:is this person, Accuracy 69.63
	Question type:can you, Accuracy 70.93
	Question type:does this, Accuracy 70.19
	Question type:is this an, Accuracy 74.03
	Question type:is there, Accuracy 70.43
	Question type:are, Accuracy 66.45
	Question type:do you, Accuracy 76.49
	Question type:is he, Accuracy 67.17
	Question type:does the, Accuracy 66.89
	Question ty

Already finish: 62000 138
	Question type:is this a, Accuracy 73.44
	Question type:is this, Accuracy 71.91
	Question type:is it, Accuracy 74.95
	Question type:none of the above, Accuracy 67.96
	Question type:is there a, Accuracy 77.62
	Question type:is the, Accuracy 67.44
	Question type:is that a, Accuracy 73.92
	Question type:are these, Accuracy 69.49
	Question type:are the, Accuracy 67.43
	Question type:could, Accuracy 68.54
	Question type:are there, Accuracy 71.26
	Question type:has, Accuracy 65.14
	Question type:are there any, Accuracy 76.15
	Question type:is this person, Accuracy 69.54
	Question type:can you, Accuracy 70.13
	Question type:does this, Accuracy 69.68
	Question type:is this an, Accuracy 74.10
	Question type:is there, Accuracy 70.01
	Question type:are, Accuracy 66.89
	Question type:do you, Accuracy 75.62
	Question type:is he, Accuracy 66.79
	Question type:does the, Accuracy 66.37
	Question type:is, Accuracy 68.48
	Question type:is the woman, Accuracy 67.11
	Question typ

Invalid caption generation for COCO_val2014_000000103499.jpg theumbrella the umbrella 
Already finish: 68000 151
	Question type:is this a, Accuracy 73.28
	Question type:is this, Accuracy 71.99
	Question type:is it, Accuracy 74.68
	Question type:none of the above, Accuracy 67.98
	Question type:is there a, Accuracy 77.73
	Question type:is the, Accuracy 67.44
	Question type:is that a, Accuracy 74.68
	Question type:are these, Accuracy 69.59
	Question type:are the, Accuracy 67.64
	Question type:could, Accuracy 68.15
	Question type:are there, Accuracy 71.51
	Question type:has, Accuracy 65.63
	Question type:are there any, Accuracy 75.97
	Question type:is this person, Accuracy 70.35
	Question type:can you, Accuracy 70.77
	Question type:does this, Accuracy 69.68
	Question type:is this an, Accuracy 74.31
	Question type:is there, Accuracy 69.86
	Question type:are, Accuracy 66.83
	Question type:do you, Accuracy 75.78
	Question type:is he, Accuracy 67.19
	Question type:does the, Accuracy 66.09
	Que

Already finish: 74000 166
	Question type:is this a, Accuracy 73.35
	Question type:is this, Accuracy 71.88
	Question type:is it, Accuracy 74.78
	Question type:none of the above, Accuracy 67.97
	Question type:is there a, Accuracy 77.39
	Question type:is the, Accuracy 67.51
	Question type:is that a, Accuracy 75.36
	Question type:are these, Accuracy 69.37
	Question type:are the, Accuracy 67.33
	Question type:could, Accuracy 68.24
	Question type:are there, Accuracy 71.36
	Question type:has, Accuracy 66.10
	Question type:are there any, Accuracy 75.79
	Question type:is this person, Accuracy 70.91
	Question type:can you, Accuracy 70.24
	Question type:does this, Accuracy 69.07
	Question type:is this an, Accuracy 74.19
	Question type:is there, Accuracy 69.96
	Question type:are, Accuracy 66.49
	Question type:do you, Accuracy 75.59
	Question type:is he, Accuracy 67.22
	Question type:does the, Accuracy 66.14
	Question type:is, Accuracy 68.70
	Question type:is the woman, Accuracy 67.46
	Question typ

Invalid caption generation for COCO_val2014_000000492395.jpg thecircle the circle 
Already finish: 80000 176
	Question type:is this a, Accuracy 73.31
	Question type:is this, Accuracy 71.95
	Question type:is it, Accuracy 74.74
	Question type:none of the above, Accuracy 68.34
	Question type:is there a, Accuracy 77.34
	Question type:is the, Accuracy 67.62
	Question type:is that a, Accuracy 75.85
	Question type:are these, Accuracy 69.13
	Question type:are the, Accuracy 67.07
	Question type:could, Accuracy 68.30
	Question type:are there, Accuracy 71.22
	Question type:has, Accuracy 66.42
	Question type:are there any, Accuracy 75.92
	Question type:is this person, Accuracy 70.52
	Question type:can you, Accuracy 70.26
	Question type:does this, Accuracy 69.20
	Question type:is this an, Accuracy 74.52
	Question type:is there, Accuracy 70.11
	Question type:are, Accuracy 66.82
	Question type:do you, Accuracy 75.15
	Question type:is he, Accuracy 66.70
	Question type:does the, Accuracy 66.04
	Questio

In [77]:
opt_pred_there=load_pkl(os.path.join(VQA_PATH,'questions/scenic_there_pred.pkl'))
print (len(opt_pred_there))

3994


In [74]:
vis=0
ood=0
img_names=defaultdict(int)
for name in names:
    info=binary_q[name]
    if info['q_type'] not in current_focus:
        continue
    vis+=1
    img_names[info['img_id']]+=1
    if name not in opt_pred_there:
        ood+=1
print (vis,ood)
print (len(img_names))

11808 7847
9099


In [55]:
obj_needed_there=load_pkl(os.path.join(
    VQA_PATH,
    'questions/obj_there_needed_opt.pkl'))
print (len(obj_needed_there))

11900


In [63]:
random.shuffle(names)
print (obj_needed_there[names[0]])

space


In [82]:
acc={q:0.0 for q in current_focus}
counter={q:0.0 for q in current_focus}
vis=0
#random.shuffle(names)
for name in names:
    info=binary_q[name]
    if info['q_type'] not in current_focus:
        continue
    vis+=1
    counter[info['q_type']]+=1
    #if vis>10:
    #    break
        
    arg=obj_needed_there[name]
    obj=stemmer.singular_noun(arg)
    if obj==False:
        obj=arg
    
    try:
        scenic_info=load_pkl(
            os.path.join(
                '../Learning-Asking/prompt-clip/scenic/there_related',
                name+'.pkl'))
    #print(scenic_info,obj)
        bbox=[info['score'] for info in scenic_info[obj] if info['score']>=0.2]
    except:
        bbox=[]
    #print (bbox)
    if len(bbox)>0:
        pred='yes'
    else:
        pred='no'
        
    if pred in info['answer']:
        score=info['scores'][info['answer'].index(pred)]
        acc[info['q_type']]+=score
        
    if vis%1000==0:
        print('Already finish:',vis)
        for q_t in counter:
            if counter[q_t]==0:
                continue
            print ('\tQuestion type:%s, Accuracy %.2f' % 
                   (q_t,acc[q_t]*100.0/counter[q_t]))
        print('\tAccuracy %.2f\n' % 
              (sum(acc.values())*100.0/sum(counter.values())))  

print ('Overall',vis)
for q_t in counter:
    if counter[q_t]==0:
        continue
    print ('\tQuestion type:%s, Accuracy %.2f, Number of instance %d' % 
           (q_t,acc[q_t]*100.0/counter[q_t],counter[q_t]))
print('\tAccuracy %.2f' % 
      (sum(acc.values())*100.0/sum(counter.values())))  

Already finish: 1000
	Question type:is there a, Accuracy 78.49
	Question type:is there, Accuracy 68.79
	Question type:are there any, Accuracy 81.67
	Question type:are there, Accuracy 75.96
	Accuracy 75.46

Already finish: 2000
	Question type:is there a, Accuracy 78.76
	Question type:is there, Accuracy 70.49
	Question type:are there any, Accuracy 77.87
	Question type:are there, Accuracy 74.09
	Accuracy 75.34

Already finish: 3000
	Question type:is there a, Accuracy 78.53
	Question type:is there, Accuracy 70.74
	Question type:are there any, Accuracy 78.77
	Question type:are there, Accuracy 73.13
	Accuracy 75.22

Already finish: 4000
	Question type:is there a, Accuracy 77.93
	Question type:is there, Accuracy 70.15
	Question type:are there any, Accuracy 79.26
	Question type:are there, Accuracy 72.00
	Accuracy 74.64

Already finish: 5000
	Question type:is there a, Accuracy 77.94
	Question type:is there, Accuracy 70.17
	Question type:are there any, Accuracy 78.28
	Question type:are there, Ac

In [245]:
random.shuffle(names)
vis=0
for name in names:
    if vis>10:
        break
    info=binary_q[name]
    ques=info['question'].lower()[:-1]
    
    parser=parsing_file[name]
    """if parser['invalid']:
        continue
    else:
        layout=parser['layout']
        ops='-'.join(step['operation'] for step in layout)
        if ops not in valid_op_seq:
            continue
        if ops not in ['select-relocate-select-and-filter']:
            continue
        print ('\n',ques,'\n',info['q_type'])
        print (qid_to_prompt[name])
        for step in layout:
            print ('\t',step['operation'],' '.join(step['argument']))"""
    img_id='COCO_val2014_'+str(info['img_id']).zfill(12)+'.jpg'
    print (ques,'\t',name,'\n',info['q_type'],info['answer'])
    print (qid_to_prompt[name])
    pred=pred_all_binary(parser,name, ques,img_id)
    print ('\tPrediction:',pred,'\n')
    vis+=1

is the train moving 	 384594009 
 is the ['no', 'yes']
['the train is moving', 'the train is not moving']
Layout
	 select train
	 filter moving
Filter checking
No exisiting scenic for COCO_val2014_000000384594.jpg train
['train'] moving
	Prediction: yes 

is the woman wearing pants 	 386204028 
 is the woman ['yes', 'no']
['the woman is wearing pants.', 'the woman is not wearing pants']
Layout
	 select pants
	 filter wearing
	 select woman
	 and 
And checking


FileNotFoundError: [Errno 2] No such file or directory: '../Learning-Asking/prompt-clip/scenic/all_binary/386204028.pkl'

In [ ]:
scene_words=['is','scene','does','are']

In [53]:
op_value={
    'and':-1,
    'filter':0,
    'relocate':0,
    'select':1
}

In [ ]:
    select-filter 28222
select 20012
select-relocate-select-and 12229
select-filter-select-and 6927
select-relocate-filter-select-and 3611
select-filter-relocate-select-and 1825
select-filter-filter-select-and 1164
    select-relocate-filter 967
        select-relocate-select-and-filter 909
select-select-and 676
        select-filter-select-and-filter 176
    select-filter-relocate-filter 118
select-relocate-relocate-select-and 110
    select-relocate-filter-filter 96
    select-filter-filter-filter 92
        select-select-and-filter 44
        select-filter-relocate-filter-filter 17
        
        
select-filter-select-filter-and 15
select-filter-filter-relocate-filter 10
select-relocate-relocate-filter 8
select-relocate-filter-filter-filter 8
select-relocate-filter-relocate 6
select-filter-filter-filter-filter 6
select-select-and-filter-filter 4
select-filter-relocate-relocate-filter 4
select-relocate-filter-relocate-filter 3
        select-select-and-relocate-filter 1
select-select-and-select-and 1

In [57]:
sort_seq=sorted(op_seq_dict.items(),
                key= lambda x:x[1],reverse=True)
valid_op_seq=[]
for op_seq,freq in sort_seq:
    length=len(op_seq.split('-'))
    counts=0
    flag=False
    for op in op_seq.split('-'):
        counts+=op_value[op]
        if counts<1:
            flag=True
    if counts!=1 or flag:
        continue
    """if length>4:
        print ('\t',op_seq,freq)
    else:
        print (op_seq,freq)"""
    print (op_seq,freq)
    valid_op_seq.append(op_seq)

select-filter 28222
select 20012
select-relocate-select-and 12229
select-filter-select-and 6927
select-relocate-filter-select-and 3611
select-filter-relocate-select-and 1825
select-filter-filter 1303
select-filter-filter-select-and 1164
select-relocate-filter 967
select-relocate-select-and-filter 909
select-select-and 676
select-filter-select-and-filter 176
select-filter-relocate-filter 118
select-relocate-relocate-select-and 110
select-relocate-filter-filter 96
select-filter-filter-filter 92
select-filter-select-and-relocate 86
select-relocate-select-and-relocate 81
select-select-and-filter 44
select-select-and-relocate 23
select-filter-relocate-filter-filter 17
select-filter-select-filter-and 15
select-filter-filter-relocate-filter 10
select-relocate-relocate-filter 8
select-relocate-filter-filter-filter 8
select-relocate-filter-relocate 6
select-filter-filter-filter-filter 6
select-select-and-filter-filter 4
select-filter-relocate-relocate-filter 4
select-relocate-filter-relocate-fi

In [123]:
random.shuffle(names)
vis=0
filter_dict=defaultdict(int)
for name in names:
    info=binary_q[name]
    ques=info['question'].lower()[:-1]
    
    parser=parsing_file[name]
    if parser['invalid']:
        continue
    else:
        layout=parser['layout']
        ops='-'.join(step['operation'] for step in layout)
        if ops not in valid_op_seq:
            continue
        if ops in ['select',
                   'select-filter-filter',
                   'select-filter-filter-filter',
                   'select-relocate-select-and',
                   'select-filter-select-and']:
            continue
        for k,step in enumerate(layout):
            if step['operation']=='filter' and k==len(layout)-1:
                filter_dict[step['argument'][0]]+=1
            
    vis+=1

In [124]:
sort_filter=sorted(filter_dict.items(),
                key= lambda x:x[1],reverse=True)
for op_seq,freq in sort_filter[:100]:
    print (op_seq,freq)

on 1212
taken 615
happy 594
sunny 528
wearing 497
moving 462
open 449
clean 449
real 407
raining 369
healthy 342
cold 317
cloudy 311
up 304
clear 296
eating 281
old 280
new 272
visible 270
wet 255
full 233
going 229
playing 216
down 212
modern 208
professional 207
good 204
sleeping 192
white 188
red 185
smiling 183
outside 165
empty 161
big 153
calm 150
flying 150
running 146
same 145
blue 139
hot 137
alone 137
safe 136
made 136
busy 134
closed 133
out 132
ready 129
large 127
ripe 124
about 123
in 122
snowing 121
crowded 120
asleep 113
alive 112
green 112
normal 109
high 109
indoors 108
black 106
standing 106
young 102
vegan 101
dirty 97
wild 96
off 95
cooked 92
nearby 90
likely 89
blurry 88
parked 88
comfortable 87
sweet 86
shining 84
messy 84
hungry 83
related 83
eaten 82
free 82
married 82
recently 81
working 81
colorful 80
used 80
there 80
looking 79
small 76
inside 76
little 76
american 75
fresh 75
awake 74
commercial 74
enough 73
fast 73
sitting 73
windy 69
cut 69
sad 68
having 6